<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.64
    Uninstalling yfinance-0.2.64:
      Successfully uninstalled yfinance-0.2.64
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-01 10:00:47
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20


-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  6.47 K
Current:  1.36 C
-------------------
Today PnL: 5.48 K (0.04%)
Current PnL: -13.49 L (-9.4%)
CY Booked + Current PnL: -6.75 L (-4.7%)
-------------------
Total profit:  4.40 L
Total loss:  -17.89 L
-------------------
Total Booked + Current PnL: 20.23 L (17.36%)
Total Booked PnL: 33.72 L (28.93%)
Curr Year Booked PnL: 6.74 L (4.94%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.08 C
Est FTT PnL: 71.92 L (52.72%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,77.08,73750.0,39301.0,649.0,-0.87,114.09,0.88,115.97,NaN,60.56,0.54,12.78,AR,ATH,ELECTRICAL
16,BLUSPRING,226.45,84.55,NaN,NaN,2.08,16331.0,-33262.0,2185.0,-1.80,-67.07,13.38,-62.66,NaN,-15.22,0.12,9.52,XY24,ATH,MISC
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,43559.0,2304.0,8503.0,1.41,5.58,19.52,26.19,1.0,0.27,0.32,1.41,XY24,ATH,IT
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,28.12,102943.0,24581.0,9450.0,-0.02,31.37,9.18,43.43,77.0,2.60,0.76,38.36,XY24,NTT,MISC
87,UJJIVANSFB,52.77,53.00,51.69,M-SC,88.54,131625.0,-10854.0,11478.0,-0.57,-7.62,8.72,0.44,243.0,-0.95,0.97,55.35,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,88.63,M-MC,40.62,226581.0,1619.0,166039.0,-0.31,0.72,73.28,74.53,185.0,0.01,1.66,31.55,XY24,BTT,STEEL
55,LALPATHLAB,2804.19,3545.00,-33.57,H-SC,39.58,153043.0,-1187.0,41934.0,-0.49,-0.77,27.40,26.42,125.0,-0.03,1.12,20.69,X40N,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2337.55,-11.71,H-SC,73.96,165510.0,840.0,68240.0,1.19,0.51,41.23,41.95,121.0,0.01,1.21,8.01,X40N,ATH,AC


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,KPIGREEN,497.21,732.16,29.90,H-SC,76.04,133056.0,7759.0,51453.0,2.59,6.19,38.67,47.25,139.0,0.15,0.98,66.46,MH,ATH,POWER
74,SIEMENS,4406.90,7969.85,-1.56,X-LC,43.75,83160.0,-27012.0,116083.0,2.30,-24.52,139.59,80.85,14.0,-0.23,0.61,22.48,AR,ATH,ELECTRICAL
30,DMART,3764.92,5201.00,-12.89,X-LC,90.62,218731.0,34250.0,36112.0,2.09,18.57,16.51,38.14,15.0,0.95,1.60,31.14,X40N,NTT,FMCG
90,VALIANTORG,512.64,838.00,-355.77,H-SC,6.25,108680.0,-24606.0,109202.0,2.01,-18.46,100.48,63.47,148.0,-0.23,0.80,82.17,XR,NTT,CHEMICALS
24,COFFEEDAY,59.14,80.00,-49.70,L-SC,100.00,73037.0,-40512.0,80560.0,1.58,-35.68,110.30,35.27,261.0,-0.50,0.54,78.76,XR,NTT,HOTELS


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,DIXON,14367.00,18940.15,-36.85,H-MC,30.21,116256.0,1320.0,35260.0,-3.01,1.15,30.33,31.83,99.0,0.04,0.85,33.24,X40N,ATH,IT
16,BLUSPRING,226.45,84.55,NaN,NaN,2.08,16331.0,-33262.0,2185.0,-1.80,-67.07,13.38,-62.66,NaN,-15.22,0.12,9.52,XY24,ATH,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,25.00,172039.0,31199.0,13299.0,-1.49,22.15,7.73,31.59,89.0,2.35,1.26,49.44,XY24,NTT,BANKS
4,ALKYLAMINE,2347.98,4546.37,7.94,X-SC,98.96,100474.0,-489.0,95018.0,-1.33,-0.48,94.57,93.63,64.0,-0.01,0.74,49.65,SR,ATH,CHEMICALS
34,HAPPSTMNDS,919.80,1496.95,-2.50,M-SC,84.38,68014.0,-28565.0,89166.0,-1.24,-29.58,131.10,62.75,209.0,-0.32,0.50,19.26,AR,ATH,IT


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,51.69,M-SC,88.54,131625.0,-10854.0,11478.0,-0.57,-7.62,8.72,0.44,243.0,-0.95,0.97,55.35,OX40N,NTT,BANKS
92,VIPIND,488.80,489.0,-215.14,H-SC,97.92,83031.0,-12285.0,12322.0,1.10,-12.89,14.84,0.04,154.0,-1.00,0.61,63.77,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-16.13,X-MC,27.08,88102.0,-14550.0,14546.0,-0.27,-14.17,16.51,-0.00,58.0,-1.00,0.65,33.93,OX40N,NTT,IT
53,KANSAINER,299.63,340.0,-67.47,H-SC,32.29,226080.0,-43587.0,79919.0,-0.38,-16.16,35.35,13.47,142.0,-0.55,1.66,13.75,XY24,NTT,PAINTS
79,SYMPHONY,1306.42,1306.0,-33.74,M-SC,5.21,141179.0,-29962.0,29902.0,0.23,-17.51,21.18,-0.03,189.0,-1.00,1.04,1.47,OX40N,NTT,DURABLES


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,237.72,NaN,NaN,1.04,43559.0,2304.0,8503.0,1.41,5.58,19.52,26.19,1.0,0.27,0.32,1.41,XY24,ATH,IT
83,TCS,3794.03,4998.00,-17.42,X-LC,42.71,269092.0,-26842.0,120742.0,-0.35,-9.07,44.87,31.73,1.0,-0.22,1.97,7.71,X40,BTT,IT
48,INFY,1461.46,2275.00,-10.27,X-LC,58.33,287742.0,26141.0,119470.0,0.36,9.99,41.52,55.67,2.0,0.22,2.11,16.71,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-17.68,X-LC,20.83,257656.0,-12691.0,47331.0,0.26,-4.69,18.37,12.81,6.0,-0.27,1.89,7.76,XY25,NTT,FMCG
61,NESTLEIND,2268.60,2755.00,-9.04,X-LC,51.04,288370.0,22944.0,33970.0,-0.02,8.64,11.78,21.44,9.0,0.68,2.12,15.38,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,UNIONBANK,123.87,163.0,-1.39,M-LC,25.00,172039.0,31199.0,13299.0,-1.49,22.15,7.73,31.59,89.0,2.35,1.26,49.44,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.40,M-LC,28.12,102943.0,24581.0,9450.0,-0.02,31.37,9.18,43.43,77.0,2.60,0.76,38.36,XY24,NTT,MISC
32,ENRIN,1377.95,2975.9,NaN,NaN,77.08,73750.0,39301.0,649.0,-0.87,114.09,0.88,115.97,NaN,60.56,0.54,12.78,AR,ATH,ELECTRICAL


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,88.63,M-MC,40.62,226581.0,1619.0,166039.0,-0.31,0.72,73.28,74.53,185.0,0.01,1.66,31.55,XY24,BTT,STEEL
54,KPIGREEN,497.21,732.16,29.90,H-SC,76.04,133056.0,7759.0,51453.0,2.59,6.19,38.67,47.25,139.0,0.15,0.98,66.46,MH,ATH,POWER
20,CAMS,3643.00,5250.99,6.54,H-SC,57.29,119546.0,17542.0,27484.0,-0.48,17.20,22.99,44.14,124.0,0.64,0.88,36.23,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,28.12,102943.0,24581.0,9450.0,-0.02,31.37,9.18,43.43,77.0,2.60,0.76,38.36,XY24,NTT,MISC
88,UNIONBANK,123.87,163.00,-1.39,M-LC,25.00,172039.0,31199.0,13299.0,-1.49,22.15,7.73,31.59,89.0,2.35,1.26,49.44,XY24,NTT,BANKS


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,2975.90,NaN,NaN,77.08,73750.0,39301.0,649.0,-0.87,114.09,0.88,115.97,NaN,60.56,0.54,12.78,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.18,M-SC,91.67,203683.0,32062.0,130011.0,-1.13,18.68,63.83,94.43,198.0,0.25,1.49,50.33,XR,NTT,DURABLES
54,KPIGREEN,497.21,732.16,29.90,H-SC,76.04,133056.0,7759.0,51453.0,2.59,6.19,38.67,47.25,139.0,0.15,0.98,66.46,MH,ATH,POWER
95,WIPRO,243.46,420.00,-3.77,M-LC,75.00,165013.0,14068.0,95394.0,0.06,9.32,57.81,72.51,69.0,0.15,1.21,13.67,XR,NTT,IT
45,INDIAMART,2327.09,4871.06,-47.14,H-SC,92.71,137413.0,14077.0,120759.0,-0.20,11.41,87.88,109.32,122.0,0.12,1.01,35.75,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-49.70,L-SC,100.00,73037.0,-40512.0,80560.0,1.58,-35.68,110.30,35.27,261.0,-0.50,0.54,78.76,XR,NTT,HOTELS
32,ENRIN,1377.95,2975.90,NaN,NaN,77.08,73750.0,39301.0,649.0,-0.87,114.09,0.88,115.97,NaN,60.56,0.54,12.78,AR,ATH,ELECTRICAL
58,MASFIN,326.60,402.39,-18.70,H-SC,60.42,92145.0,-5835.0,28574.0,-0.29,-5.96,31.01,23.21,137.0,-0.20,0.68,33.64,XR,ATH,FINANCE
4,ALKYLAMINE,2347.98,4546.37,7.94,X-SC,98.96,100474.0,-489.0,95018.0,-1.33,-0.48,94.57,93.63,64.0,-0.01,0.74,49.65,SR,ATH,CHEMICALS
3,ADANIPORTS,1103.69,1583.00,2.40,M-LC,28.12,102943.0,24581.0,9450.0,-0.02,31.37,9.18,43.43,77.0,2.60,0.76,38.36,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,COFFEEDAY,59.14,80.00,-49.70,L-SC,100.00,73037.0,-40512.0,80560.0,1.58,-35.68,110.30,35.27,261.0,-0.50,0.54,78.76,XR,NTT,HOTELS
4,ALKYLAMINE,2347.98,4546.37,7.94,X-SC,98.96,100474.0,-489.0,95018.0,-1.33,-0.48,94.57,93.63,64.0,-0.01,0.74,49.65,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-215.14,H-SC,97.92,83031.0,-12285.0,12322.0,1.10,-12.89,14.84,0.04,154.0,-1.00,0.61,63.77,OX40N,NTT,MISC
52,JPPOWER,18.73,26.20,-30.97,L-SC,96.88,138965.0,-2877.0,59449.0,0.27,-2.03,42.78,39.88,259.0,-0.05,1.02,40.83,XY24,NTT,POWER
72,SFL,1016.35,1287.00,41.65,H-SC,95.83,187102.0,-75116.0,144948.0,-0.20,-28.65,77.47,26.63,144.0,-0.52,1.37,16.33,XY24,NTT,MISC


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.96
1,25,42.00
2,50,70.39


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.50
LC    34.18
MC    21.37
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.48
X40      12.24
XY25     12.09
X40N     11.74
XR       10.17
AR        8.30
OX40N     8.17
MH        1.73
X5K       1.47
X200      1.41
SR        1.23
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.67
M-SC    16.00
H-LC    14.18
X-LC    12.26
H-MC     9.45
X-MC     7.53
M-LC     6.67
X-SC     4.36
M-MC     4.02
L-SC     2.47
L-LC     1.07
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.18,-1.09,34.89
IT,12.09,-10.29,64.45
FINANCE,7.97,-11.15,51.38
BANKS,7.92,-5.60,48.68
MISC,6.67,-14.58,67.75
PAINTS,5.65,-13.36,35.70
ELECTRICAL,5.52,2.24,43.42
HEALTHCARE,4.48,-5.50,36.31
AUTO,4.36,-13.92,64.02


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2709460.0,27
XR,1073764.0,13
AR,1012856.0,10
X40,648045.0,9
X40N,563502.0,11
XY25,471429.0,8
OX40N,319406.0,11
SR,176203.0,2
MH,80468.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1968567.0,20
M-SC,1763167.0,21
H-LC,608410.0,12
H-MC,544928.0,8
X-LC,502852.0,7
X-MC,424664.0,6
M-MC,421240.0,3
X-SC,323964.0,5
M-LC,264995.0,5


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      854739.0      6
M-SC       XY24      813007.0      7
H-SC       AR        519913.0      3
M-SC       XR        333081.0      4
M-MC       XY24      310712.0      2
H-MC       XY24      299921.0      3
X-LC       X40       240212.0      2
H-SC       X40N      232827.0      4
           XR        217017.0      3
H-LC       X40N      201501.0      4
M-SC       AR        181072.0      2
           OX40N     172441.0      5
M-LC       XY24      157576.0      3
H-LC       X40       156872.0      3
X-MC       X40       149985.0      2
L-SC       XR        141961.0      2
X-SC       XR        123183.0      1
X-LC       AR        116083.0      1
M-MC       XR        110528.0      1
X-LC       XY25      110445.0      3
M-SC       XY25      109988.0      1
X-MC       XY25      105350.0      1
           XY24       96981.0      1
M-LC       XR         95394.0      1
X-SC       SR         95018.0      1
H-SC       OX40N      92618.0      3
H-MC       XY25       89421.0      1
M-SC       SR         81185.0      1
H-LC       AR         72971.0      1
M-SC       X40        72393.0      1
H-LC       X200       68578.0      1
           X5K        68063.0      2
X-SC       XY24       65962.0      1
H-MC       AR         62728.0      1
L-SC       XY24       59449.0      1
           AR         59440.0      1
X-MC       X40N       57802.0      1
L-MC       XR         52600.0      1
H-SC       MH         51453.0      1
L-LC       XY25       44200.0      1
H-LC       XY24       40425.0      1
X-SC       OX40N      39801.0      2
X-LC       X40N       36112.0      1
H-MC       X40N       35260.0      1
           MH         29015.0      1
           X40        28583.0      1
X-MC       OX40N      14546.0      1
M-LC       XY25       12025.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 37.0 seconds
